## Topic : Imagine you are a manager of a Team!

<img src='https://cdn.images.express.co.uk/img/dynamic/67/590x/Mourinho-Lampard-City-513573.jpg' width=500>


### Which team do you support?

### Who is Best Manager for you?

For me, 

**"Jose Mourinho" Chelsea 2004-2005**

Let's Imagine If you were them!

I have made a simple recommender system for various decision making as a manager

#### 1. Recommend players
**case1 : key player leaves team.**
<img src='https://i.pinimg.com/originals/dd/b9/77/ddb977b04f4cfd1de611700f462fff57.jpg' width=300>
<br>
**case2 : players similar to my team.**
#### 2. Recommend Teams
**You'd like to move to other teams.**

# 1. Load library

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


import os

from plotly.offline import init_notebook_mode, iplot, plot
from plotly.subplots import make_subplots
import plotly.express as px
import plotly as py
init_notebook_mode(connected=True)
import plotly.graph_objs as go

from sklearn.neighbors import NearestNeighbors

pd.set_option('display.max_columns', None)

import warnings
warnings.filterwarnings('ignore')

In [ ]:
pd.set_option('precision', 0)

In [ ]:
train = pd.read_csv('/kaggle/input/fifa19/data.csv')

# 2. Data preprocessing
- string data type to numeric type

In [ ]:
train['Release Clause'] = train['Release Clause'].str.slice(1,-1).astype(float)

train['Wage'] = train['Wage'].str.slice(1,-1)
train['Wage'] = np.where(train['Wage'] =='', '0', train['Wage'])
train['Wage'] = train['Wage'].astype(int)

train['Contract Valid Until'] = train['Contract Valid Until'].fillna('X')
train['Contract Valid Until']= [np.nan if 'X' in a else int(a) for a in train['Contract Valid Until'].str.slice(-4)]

train['Foot'] = pd.factorize(train['Preferred Foot'])[0]

In [ ]:
rename_dict = {'Release Clause':'BuyOut(M €)',
               'Wage':'Wage(K €)',
               'Contract Valid Until':'Contract Expire'}

train.rename(columns=rename_dict, inplace=True)

In [ ]:
# feet to cm
train['Height'] = train['Height'].str.replace("'",'.').astype(float) * 30.48
# pound to kg
train['Weight'] = train['Weight'].str.replace("lbs",'').astype(float) / 2.205

train[['Height','Weight']].head()

# 3. Recommender

## 3-1. recommend players - Key player leaves team

- I will calculate similarity among players using KNN

In [ ]:
field_cols = ['Foot', 'Weak Foot', 'Skill Moves', 'Height', 'Weight', 'Crossing', 
             'Finishing', 'HeadingAccuracy', 'ShortPassing', 'Volleys', 'Dribbling',
             'Curve', 'FKAccuracy', 'LongPassing', 'BallControl', 'Acceleration',
             'SprintSpeed', 'Agility', 'Reactions', 'Balance', 'ShotPower',
             'Jumping', 'Stamina', 'Strength', 'LongShots', 'Aggression',
             'Interceptions', 'Positioning', 'Vision', 'Penalties', 'Composure',
             'Marking', 'StandingTackle', 'SlidingTackle']

In [ ]:
train[field_cols].head()

In [ ]:
tr = train[['ID','Name']+field_cols].dropna()
id_name = train[['Name','ID']].set_index('ID')['Name'].to_dict()

In [ ]:
NUM_RECOM = 10

X = tr[field_cols].dropna().values
nbrs = NearestNeighbors(n_neighbors=NUM_RECOM+1, algorithm='ball_tree').fit(X)
dist, rank = nbrs.kneighbors(X)

similar_df = pd.DataFrame(columns=[f'rank_{i}'for i in range(1,NUM_RECOM+1)],
                          index=tr['ID'].values,
                          data=rank[:,1:])
dist_df = pd.DataFrame(columns=[f'rank_{i}'for i in range(1,NUM_RECOM+1)],
                       index=tr['ID'].values,
                       data=dist[:,1:])


for cols in list(similar_df):
    tg_col = similar_df[cols]
    new_value = tr['ID'].iloc[tg_col].tolist()
    similar_df[cols] = new_value

In [ ]:
def similar_player(similar_df, dist_df, player_id):
    player_id = int(player_id)
    player_name = tr[tr['ID'] == player_id]['Name'].values[0]
    
    ## Bar chart
    Xaxis = 1/(1+dist_df.loc[player_id].values[::-1])
    Yaxis = similar_df.loc[player_id].map(id_name).values[::-1]

    fig = go.Figure(go.Bar(
                x=Xaxis,
                y=Yaxis,
                orientation='h'))
    
    fig.update_layout(title_text='Players similar to  " '+str(player_name)+' "')
    fig.show()

    ## Table display
    sim_ply = similar_df.loc[player_id]
    display_col = ['Name','Age','Nationality','Overall','Potential', 'Club', 'Preferred Foot','Weak Foot', 
               'Contract Expire', 'Wage(K €)','BuyOut(M €)']

    display_df = pd.DataFrame({'ID':similar_df.loc[player_id]})\
    .merge(train[['ID']+display_col], how='left', on='ID')[display_col]


    display(display_df.style.highlight_max(color='lightgreen').highlight_min(color='#cd4f39'))


In [ ]:
similar_player(similar_df, dist_df, 183277)

From above graph, we can see Neymar Jr is a player most simliar to E. Hazard.

## 3-2. recommend players - a player fitting my team

1. remapping position into my custom position based on my knowledge
    - DF : side_df, cent_df
    - MF : side_mf, cent_mf
    - FW : side_fw, cent_fw

2. select columns for each custom positions based on my knowledge too. 

3. Next, calculate similarity between your team and other players using KNN


In [ ]:
position_remap = {'LWB':'side_df',
                  'LB':'side_df',
                  'RB':'side_df',
                  'RWB':'side_df',

                  'LCB':'cent_df',
                  'CB':'cent_df',
                  'RCB':'cent_df',
                  
                  'LDM':'cent_md',
                  'CDM':'cent_md',
                  'RDM':'cent_md',

                  'LCM':'cent_md',
                  'CM':'cent_md',
                  'RCM':'cent_md',
                  'CAM':'cent_md',

                  'LW':'side_md',
                  'LM':'side_md',
                  'RM':'side_md',
                  'RW':'side_md',
                  'LAM':'side_md',
                  'RAM':'side_md',

                  'LF':'side_fw',
                  'LS':'side_fw', 
                  'RS':'side_fw',
                  'RF':'side_fw',

                  'CF':'cent_fw',
                  'ST':'cent_fw'}  

In [ ]:
train['position_group'] = train['Position'].map(position_remap)
select_club = train['Club'].head(5000).unique()

mask = (train['Club'].isin(select_club)) & (train['Club'].notnull())
tr = train[mask]

In [ ]:
cols_by_position = dict()
cols_by_position['cent_df'] = ['Jumping','StandingTackle', 'StandingTackle', 'Interceptions', 'Marking', 
                               'HeadingAccuracy', 'Composure', 'ShortPassing', 'Strength', 
                               'LongPassing', 'Vision', 'SprintSpeed']

cols_by_position['side_df'] = ['Dribbling', 'Crossing', 'StandingTackle', 'SlidingTackle', 'Reactions', 'BallControl', 
                               'SprintSpeed', 'Agility', 'ShortPassing', 'LongPassing', 
                               'Stamina', 'Vision', 'Acceleration']

cols_by_position['cent_md'] = ['Positioning', 'Dribbling', 'Jumping', 'Composure', 'Crossing', 'StandingTackle', 
                               'SlidingTackle', 'ShortPassing', 'LongPassing', 'Marking',
                               'Stamina', 'Vision', 'LongShots', 'ShotPower']

cols_by_position['side_md'] = ['Positioning', 'Dribbling', 'Crossing', 'Reactions', 'BallControl', 'SprintSpeed', 
                               'Agility', 'ShortPassing', 'LongPassing', 'Stamina', 
                               'LongShots', 'ShotPower', 'Vision', 'Acceleration']

cols_by_position['cent_fw'] = ['Positioning', 'Dribbling', 'Jumping', 'HeadingAccuracy', 'Composure', 'Reactions',
                               'BallControl', 'SprintSpeed', 'ShortPassing', 'Balance', 'Strength', 'Finishing',
                               'ShotPower', 'Vision', 'Acceleration', 'Volleys']

cols_by_position['side_fw'] = ['Positioning', 'Dribbling', 'Composure', 'Crossing', 'Reactions', 'BallControl', 
                               'Curve', 'SprintSpeed',  'Agility', 'ShortPassing', 'Stamina', 
                               'LongShots', 'ShotPower', 'Vision', 'Acceleration']

In [ ]:
def player_sim_team(team, position, NUM_RECOM, AGE_upper_bound):
    print('position :', position)
    target_cols = cols_by_position[position]
    
    # team stats
    team_stats = tr.query('position_group == @position and Club == @team').head(3)[target_cols].mean(axis=0)
    team_stats_np = team_stats.values
    
    # player stats by each position
    ply_stats = tr.query('position_group == @position and Club != @team and Age <= @AGE_upper_bound')[['ID']+target_cols]
    ply_stats_np = ply_stats[target_cols].values
    X = np.vstack((team_stats_np, ply_stats_np))
    
    ## KNN
    nbrs = NearestNeighbors(n_neighbors=NUM_RECOM+1, algorithm='ball_tree').fit(X)
    dist, rank = nbrs.kneighbors(X)
    
    
    ## Bar chart
    Xaxis = 1 / (1 + dist[0,1:][::-1])
    Yaxis = ply_stats.iloc[rank[0,1:]]['ID'].map(id_name).values[::-1]

    fig = go.Figure(go.Bar(
                x=Xaxis,
                y=Yaxis,
                orientation='h'))

    fig.update_layout(title_text='Players similar to  " '+str(team)+' "')
    fig.show()

    
    ## Table display
    display_col = ['Name','Age','Nationality','Overall','Potential', 'Club', 'Preferred Foot','Weak Foot', 
                   'Contract Expire', 'Wage(K €)','BuyOut(M €)']

    indice = ply_stats.iloc[rank[0,1:]].index.tolist()
    display_df = tr.loc[indice,]

    print("Player's Information")
    display(display_df[display_col].style.highlight_max(color='lightgreen').highlight_min(color='#cd4f39'))

    print("Player's Stats")
    display(display_df[['Name']+target_cols].style.highlight_max(color='lightgreen').highlight_min(color='#cd4f39'))

In [ ]:
player_sim_team(team = 'Chelsea', position = 'cent_fw', NUM_RECOM = 10, AGE_upper_bound = 25)